# Применение методов машинного обучения - линейная регрессия

В этом примере рассматривается одно из применений простейшего метода машинного обучения - линейной регрессии для предсказания будущих цен открытия инструментов.

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

from qnt.data import ds, load_assets, load_data, write_output, restore_origin_data
from qnt.stepper import test_strategy
from qnt.stats import calc_stat
from qnt.graph import make_plot, make_plot_double, make_plot_filled

from sklearn.linear_model import LinearRegression

In [ ]:
def liquid_assets_list(data):
    liquid_assets = data.loc[:, "is_liquid"].to_pandas().iloc[0, :] == 1
    return liquid_assets[liquid_assets == True].index

In [ ]:
def sign_slope(X_train, y_train):
    model = LinearRegression()
    model.fit(X_train.reshape(-1,1), y_train.values.reshape(-1,1))
    return np.sign(model.coef_[0][0])

In [ ]:
assets = load_assets()
assets_names = [x['id'] for x in assets]

data = load_data(min_date="2010-01-01", max_date="2016-12-31",
                 dims=('time', 'field', 'asset'), 
                 assets=assets_names)

## Реализация

Рассмотрим первые 250 загруженных цен акции Apple. Разделим их на самую первую цену и с помощью линейной регрессии оценим трендовость цен.

In [ ]:
model = LinearRegression()

x = np.arange(250)

prices = data.loc[:, "close", "NASDAQ:AAPL"].to_series().iloc[::-1]
y = prices.iloc[0:250] / prices.iloc[0]

model.fit(x.reshape(-1,1), y.values.reshape(-1,1))
z = np.arange(250)*model.coef_[0][0] + model.intercept_

make_plot_double(x, y, z, name1="markers", name2="lines")

Здесь мы видим возрастающий тренд цен. Будем считать, что следующие 20 дней цена будет расти, и вес на следующие 20 дней приравняем к знаку наклона. Через 20 дней снова рассмотрим предыдущие 250 дней, снова построим регрессию и обновим веса.

In [ ]:
wforward = 20
wback = 250
weights = None

def step(data):
    global weights
       
    if (data.shape[0] - init_data_length - 1) % wforward == 0:        
        prices = data.loc[:, "open"].to_pandas().iloc[(wback-1)::-1,:].dropna(axis=1)
        y_train = prices.div(prices.iloc[-1, :], axis=1)
        weights = y_train.apply(lambda column: sign_slope(np.arange(wback), column), axis=0)
    
    liquid_assets = liquid_assets_list(data)  
    weights_adj = weights[liquid_assets]
    
    weights_sum = abs(weights_adj)
    weights_norm = weights_adj / weights_sum
    assets = weights_norm.index
    
    return xr.DataArray(
        weights_norm.values,
        dims = [ds.ASSET],
        coords = {ds.ASSET:assets}
    )

init_data_length = 250

output = test_strategy(data, step=step, init_data_length=init_data_length)

# Результаты

In [ ]:
stat = calc_stat(data, output, slippage_factor=0.05)
print(stat.to_pandas().tail())

# Построение графиков

In [ ]:
performance = stat.to_pandas()['equity']
make_plot_filled(performance.index, performance, name="PnL")

In [ ]:
UWchart = stat.to_pandas()['underwater']
make_plot_filled(UWchart.index, UWchart, color="darkred", name="underwater chart")

In [ ]:
SRchart = stat.to_pandas()['sharpe_ratio'].iloc[20:]
make_plot_filled(SRchart.index, SRchart, color="#F442C5", name="rolling SR")

In [ ]:
#data = load_data(min_date="2015-01-01", dims=('time', 'field', 'asset'), assets=assets_names)
#output = test_strategy(data, step=step, init_data_length=init_data_length)
write_output(output)